In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',500)

##### Reading visit data

In [3]:
df = pd.read_csv('./Data/VisitorLogsData.csv')

In [4]:
df.shape

(6588000, 9)

In [5]:
df['UserID'].nunique()

34050

In [6]:
df.isnull().sum()

webClientID            0
VisitDateTime     658915
ProductID         527137
UserID           5937305
Activity          889446
Browser                0
OS                     0
City             2165831
Country           397693
dtype: int64

##### Dropping null values in most significant column

In [7]:
df.dropna(subset =['UserID'],inplace=True)

In [8]:
df.shape

(650695, 9)

In [9]:
df[~df['VisitDateTime'].isna()]['UserID'].nunique()

33357

In [13]:
df['VisitDateTime'].mode()[0]

'2018-05-09 13:44:40.741'

In [14]:
df['VisitDateTime'].fillna(df['VisitDateTime'].mode()[0],inplace=True)

##### Cleaning VisitDateTime Column

In [15]:
from datetime import datetime, timedelta

In [16]:
def unix_or_dt(a):
    my_format = '%Y-%m-%d %H:%M:%S.%f'
    try:
        return datetime.strptime(str(a), '%Y-%m-%d %H:%M:%S.%f').strftime(my_format)
    except:
        return datetime.utcfromtimestamp(int(a[:10])).strftime(my_format)

In [17]:
df['VisitDateTimeFormatted'] = df['VisitDateTime'].apply(lambda row: unix_or_dt(row))  

In [18]:
df[['VisitDateTime','VisitDateTimeFormatted']].head(10)

,VisitDateTime,VisitDateTimeFormatted
14,1526366895249000000,2018-05-15 06:48:15.000000
21,1527058921790000000,2018-05-23 07:02:01.000000
23,2018-05-10 06:28:53.391,2018-05-10 06:28:53.391000
24,2018-05-08 12:40:02.153,2018-05-08 12:40:02.153000
33,2018-05-11 15:35:43.689,2018-05-11 15:35:43.689000
50,2018-05-19 00:02:31.347,2018-05-19 00:02:31.347000
54,2018-05-19 04:51:45.337,2018-05-19 04:51:45.337000
61,2018-05-07 05:54:39.408,2018-05-07 05:54:39.408000
68,2018-05-23 09:44:44.023,2018-05-23 09:44:44.023000
69,2018-05-13 13:17:03.751,2018-05-13 13:17:03.751000


In [19]:
df['VisitDateTimeFormatted'].dtype

dtype('O')

In [20]:
df['VisitDateTimeFormatted'] = pd.to_datetime(df['VisitDateTimeFormatted'])

##### Restricting data for last 21 days

In [21]:
max_date = '2018-05-28'

In [22]:
my_format = '%Y-%m-%d'
datetime.strptime(str(max_date), '%Y-%m-%d').strftime(my_format)

'2018-05-28'

In [23]:
max_date = pd.to_datetime(max_date)

In [24]:
type(max_date)

pandas._libs.tslibs.timestamps.Timestamp

In [25]:
max_date -timedelta(days=1)

Timestamp('2018-05-27 00:00:00')

In [26]:
max_date -timedelta(days=21)

Timestamp('2018-05-07 00:00:00')

In [27]:
visit_df = df[(df['VisitDateTimeFormatted']>= max_date -timedelta(days=21)) &  (df['VisitDateTimeFormatted']<max_date)] 

In [28]:
visit_df.shape

(650695, 10)

In [29]:
print('Minimum Date',visit_df['VisitDateTimeFormatted'].min())
print('Maximum Date',visit_df['VisitDateTimeFormatted'].max())

Minimum Date 2018-05-07 00:00:01.419000
Maximum Date 2018-05-27 23:59:23.729000


##### Merge User and Visit Data

In [30]:
user_df = pd.read_csv('./Data/userTable.csv')

In [31]:
visit_user_merge = pd.merge(visit_df,user_df,on='UserID')

In [32]:
visit_user_merge.head()

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country,VisitDateTimeFormatted,Signup Date,User Segment
0,WI100000112772,1526366895249000000,Pr100017,U106593,click,Chrome Mobile,Android,,,2018-05-15 06:48:15.000,2017-09-10 15:48:09.451327+00:00,B
1,WI100000112772,2018-05-16 06:53:23.737,Pr100241,U106593,pageload,Chrome Mobile,Android,,,2018-05-16 06:53:23.737,2017-09-10 15:48:09.451327+00:00,B
2,WI100000112772,2018-05-15 06:47:49.239,Pr100017,U106593,click,Chrome Mobile,Android,,,2018-05-15 06:47:49.239,2017-09-10 15:48:09.451327+00:00,B
3,WI100000112772,2018-05-16 06:54:26.424,pr100901,U106593,PAGELOAD,Chrome Mobile,Android,,,2018-05-16 06:54:26.424,2017-09-10 15:48:09.451327+00:00,B
4,WI100000112772,2018-05-15 06:47:36.691,Pr100017,U106593,CLICK,Chrome Mobile,Android,,,2018-05-15 06:47:36.691,2017-09-10 15:48:09.451327+00:00,B


In [33]:
columns = ['ProductID','UserID','Activity','Browser','OS']
for i in columns:
    visit_user_merge[i] = visit_user_merge[i].str.upper()

In [34]:
visit_user_merge['Activity'].value_counts()

CLICK       395613
PAGELOAD    167325
Name: Activity, dtype: int64

In [35]:
visit_user_merge['Browser'].value_counts()

CHROME                        498641
CHROME MOBILE                  85057
FIREFOX                        33831
SAFARI                         21076
OPERA                           3621
MOBILE SAFARI                   2954
SAMSUNG INTERNET                1974
CHROME MOBILE IOS               1406
EDGE                             408
FIREFOX MOBILE                   397
CHROME MOBILE WEBVIEW            328
OPERA MOBILE                     275
COC COC                          272
FACEBOOK                         147
YANDEX BROWSER                   114
AMAZON SILK                       45
UC BROWSER                        40
APPLE MAIL                        37
MOBILE SAFARI UI/WKWEBVIEW        22
QQ BROWSER                        18
CHROMIUM                          15
MAXTHON                            9
FIREFOX IOS                        8
Name: Browser, dtype: int64

In [36]:
visit_user_merge['OS'].value_counts()

WINDOWS      463639
ANDROID       89184
MAC OS X      55260
LINUX         24020
UBUNTU        13337
IOS            4390
CHROME OS       709
FEDORA          141
TIZEN            15
Name: OS, dtype: int64

In [37]:
visit_user_merge['User Segment'].value_counts()

B    321089
C    206367
A    123239
Name: User Segment, dtype: int64

##### Submission Format

In [38]:
sub_df = pd.read_csv('./Data/sample_submission_M7Vpb9f.csv')

In [39]:
sub_df.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U100002,1,1,1,P12345,OS12345,P12345,1,1
1,U100003,1,1,1,P12345,OS12345,P12345,1,1
2,U100004,1,1,1,P12345,OS12345,P12345,1,1
3,U100005,1,1,1,P12345,OS12345,P12345,1,1
4,U100006,1,1,1,P12345,OS12345,P12345,1,1


In [40]:
max_date - timedelta(days=7)

Timestamp('2018-05-21 00:00:00')

In [41]:
visit_user_merge.head(2)

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country,VisitDateTimeFormatted,Signup Date,User Segment
0,WI100000112772,1526366895249000000,PR100017,U106593,CLICK,CHROME MOBILE,ANDROID,,,2018-05-15 06:48:15.000,2017-09-10 15:48:09.451327+00:00,B
1,WI100000112772,2018-05-16 06:53:23.737,PR100241,U106593,PAGELOAD,CHROME MOBILE,ANDROID,,,2018-05-16 06:53:23.737,2017-09-10 15:48:09.451327+00:00,B


In [42]:
visit_user_merge['VisitDate'] = visit_user_merge['VisitDateTimeFormatted'].dt.date

In [43]:
visit_user_merge['VisitDate'].head()

0    2018-05-15
1    2018-05-16
2    2018-05-15
3    2018-05-16
4    2018-05-15
Name: VisitDate, dtype: object

In [44]:
print(visit_user_merge.shape)
print(visit_user_merge['UserID'].nunique())

(650695, 13)
34050


In [45]:
visit_user_merge['Signup Date'].dtype

dtype('O')

In [46]:
visit_user_merge['Signup Date'] = pd.to_datetime(visit_user_merge['Signup Date'], 
                                       format='%Y-%m-%d %H:%M:%S', 
                                       errors='coerce')

In [47]:
visit_user_merge[['UserID','Signup Date']].head()

,UserID,Signup Date
0,U106593,2017-09-10 15:48:09.451327+00:00
1,U106593,2017-09-10 15:48:09.451327+00:00
2,U106593,2017-09-10 15:48:09.451327+00:00
3,U106593,2017-09-10 15:48:09.451327+00:00
4,U106593,2017-09-10 15:48:09.451327+00:00


In [48]:
datetime.now()

datetime.datetime(2021, 7, 2, 15, 55, 0, 74911)

In [49]:
visit_user_merge['Signup Date'][0]

Timestamp('2017-09-10 15:48:09.451327+0000', tz='UTC')

In [50]:
# now_asia = now_utc.astimezone(timezone('Asia/Kolkata'))
# print(now_asia.strftime(format))

##### TimeZone UTC to local time

In [51]:

visit_user_merge['Signup Date'] = visit_user_merge['Signup Date'].dt.tz_localize(None)

In [52]:
(datetime.now() - visit_user_merge['Signup Date'][0]).days

1391

In [53]:
visit_user_merge['VisitDate'] = pd.to_datetime(visit_user_merge['VisitDate'])

In [54]:
visit_user_merge['VisitDate'].dtype

dtype('<M8[ns]')

In [55]:
User_Id = []
No_of_days_last7_lst = []
No_of_Products_last15_lst = []
vintage_days_lst = []
freq_prod_lst = []
Most_Active_OS_lst = []
freq_prod_payload_lst = []
page_load_cnt_lst = []
click_cnt_lst = []
user_id =visit_user_merge['UserID'].unique().tolist()
print(len(user_id))
count = 0
for i in user_id:
    ## No of days visited last 7 days
    User_Id.append(i)
    print(count)
    df_ = visit_user_merge[visit_user_merge['UserID']==i]
    df_.reset_index(inplace=True,drop=True)
    df_1 = df_[(df_['VisitDate']>max_date - timedelta(days=7)) & (df_['VisitDate']<max_date)]
    if df_1.shape[0] != 0:
        No_of_days_last7 = df_1['VisitDate'].nunique()
        No_of_days_last7_lst.append(No_of_days_last7)
    else:
        No_of_days_last7_lst.append(0)
    
    ## No of products last 15 days
    df_2 = df_[(df_['VisitDate']>max_date - timedelta(days=15)) & (df_['VisitDate']<max_date)]
    if df_2.shape[0] != 0:
        No_of_Products_last15 = df_2['ProductID'].nunique()
        No_of_Products_last15_lst.append(No_of_Products_last15)
    else:
        No_of_Products_last15_lst.append(0)
        
    ## Vintage days
    sign_dt = df_['Signup Date'][0]
    vintage_days = (max_date - sign_dt).days
    vintage_days_lst.append(vintage_days)
    
    ## Most frequently viewed product
    df_3 = df_[(df_['VisitDate']>max_date - timedelta(days=15)) & (df_['VisitDate']<max_date) & (~ df_['ProductID'].isna())]
    df_3.sort_values(by=['VisitDateTimeFormatted'],ascending = False,inplace=True)
    
    if df_3.shape[0] != 0:
        print('******************************')
        print(df_3['ProductID'].mode())
        print(df_3.shape)
        freq_prod = df_3['ProductID'].mode()[0]
        freq_prod_lst.append(freq_prod)  
    else:
        freq_prod_lst.append('Product101')
        
    ## Most frequently used OS
    Most_Active_OS = df_['OS'].mode()[0]
    Most_Active_OS_lst.append(Most_Active_OS)
    
    ## Recently Viewed Product
    df_4 = df_[(df_['Activity'] == 'PAGELOAD') & (~ df_['ProductID'].isna())]
    df_4.sort_values(['VisitDateTimeFormatted'],ascending = False,inplace=True)
    df_4.reset_index(inplace=True,drop=True)
    if df_4.shape[0] != 0:
#         print(df_4['ProductID'].value_counts)
#         print('******************************')
        
        freq_prod_payload = df_4['ProductID'][0]
        freq_prod_payload_lst.append(freq_prod_payload)  
    else:
        freq_prod_payload_lst.append('Product101')
    
    ## Pageloads last 7 days
    count = count+1
    df_5 = df_[(df_['VisitDate']>max_date - timedelta(days=7)) & (df_['VisitDate']<max_date)]
    if df_5.shape[0] != 0:
        page_load_cnt = df_5[df_5['Activity']=='PAGELOAD'].shape[0]
        click_cnt = df_5[df_5['Activity']=='CLICK'].shape[0]
        page_load_cnt_lst.append(page_load_cnt)
        click_cnt_lst.append(click_cnt)

    else:
        page_load_cnt_lst.append(0)
        click_cnt_lst.append(0)
    
#     if count == 20:
#         break
    

final_df = pd.DataFrame()
final_df['UserID'] = User_Id
final_df['No_of_days_Visited_7_Days'] = No_of_days_last7_lst
final_df['No_Of_Products_Viewed_15_Days'] = No_of_Products_last15_lst
final_df['User_Vintage'] = vintage_days_lst
final_df['Most_Viewed_product_15_Days'] = freq_prod_lst
final_df['Most_Active_OS'] = Most_Active_OS_lst
final_df['Recently_Viewed_Product'] = freq_prod_payload_lst
final_df['Pageloads_last_7_days'] = page_load_cnt_lst
final_df['Clicks_last_7_days'] = click_cnt_lst

34050
0
******************************
0    PR100017
dtype: object
(32, 13)
1


C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


******************************
0    PR101030
dtype: object
(18, 13)
2
3
4
5
******************************
0    PR100340
dtype: object
(36, 13)
6
******************************
0    PR100060
dtype: object
(535, 13)
7
******************************
0    PR100017
dtype: object
(6, 13)
8
******************************
0    PR101042
dtype: object
(580, 13)
9
******************************
0    PR101042
dtype: object
(220, 13)
10
******************************
0    PR101964
dtype: object
(80, 13)
11
******************************
0    PR100573
dtype: object
(11, 13)
12
******************************
0    PR100515
1    PR102361
dtype: object
(36, 13)
13
******************************
0    PR100152
dtype: object
(54, 13)
14
******************************
0    PR101614
dtype: object
(3, 13)
15
******************************
0    PR100339
dtype: object
(92, 13)
16
******************************
0    PR100784
dtype: object
(118, 13)
17
******************************
0    PR101042
dtype: object
(

In [56]:
final_df.head()

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U106593,2,9,259,PR100017,ANDROID,PR103384,4,5
1,U108297,2,6,1097,PR101030,WINDOWS,PR100108,9,4
2,U132443,0,0,240,Product101,WINDOWS,PR100070,0,0
3,U134616,0,0,447,Product101,WINDOWS,PR100495,0,0
4,U130784,0,0,262,Product101,CHROME OS,PR102323,0,0


In [57]:
final_df.to_csv('Submission_Jobathon_MohanaPriya.csv')

In [58]:
len(sub_df.columns)

9

In [59]:
sub_df.shape

(34050, 9)

In [60]:
len(final_df.columns)

9

In [61]:
final_df.shape

(34050, 9)